Request from 1/27/2021

data from 1950 to 2019 as monthly sum in netcdf format for:
- Precipitation (input)
- actual ET
- recharge (deep drainage)
- surface runoff

Precipitation (input) is the only one that needs to be summed up, the output data is already summed up.

outputs are based on "run XX"

code is based on TonyB script "00-Create-Wzell-ET-Outputs-Sums-PPT-Only-its-SPECIAL" and "01-create-netcdfs-and-tarballs-cleanup.ipynb
"

In [1]:
%env CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt

env: CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt


In [2]:
import os
from enduserLib.sum_mydata import monthly_sum
from enduserLib.xr_netcdf import xr_build_cube_concat_ds
from enduserLib.tony_stuff import tony_mkdir

In [3]:
def create_s3_list_of_days(main_prefix, year, input_name='PPT_'):
    the_list = []
    for i in range(1,366):
        month = f'{i:03d}'
        file_object = main_prefix +os.sep+ str(year) +os.sep+ input_name + str(year) + month + '.tif'
        the_list.append(file_object)
    return the_list

In [4]:
# to run just 1 year

#main_prefix = 's3://dev-et-data/in/DelawareRiverBasin/PPT'
#year = 2003
#output_name = 'PPT_'
#ppt_list = create_s3_list_of_days(main_prefix, year, output_name)

In [5]:
def _mkdir(directory):
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

# Create PPT Sum Tifs

In [6]:
_mkdir('./junkboxpptsum')

In [7]:
main_prefix = 's3://dev-et-data/in/DelawareRiverBasin/PPT'
out_dir = './junkboxpptsum'
input_name = 'PPT_'
out_product = 'ppt_'


# 2000-2018 was already created for a previous request
# just do 1950 - 1999
# monthly function comes from http://10.12.68.246/user/wzell/tree/opt/enduser/enduserLib/enduserLib
print('Data is outputted to: {}'.format(out_dir))
for year in range(1966,2019):
    file_list = create_s3_list_of_days(main_prefix, year, input_name)
    monthly_sum(file_list, out_dir, out_product, year)

Data is outputted to: ./junkboxpptsum
Month summed up is: 1
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966001.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966002.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966003.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966004.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966005.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966006.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966007.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966008.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966009.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966010.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966011.tif
reading file ... s3://dev-et-data/in/DelawareRive

reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966101.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966102.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966103.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966104.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966105.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966106.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966107.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966108.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966109.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966110.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966111.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966112.tif
reading file ... s3://dev-et

reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966201.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966202.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966203.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966204.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966205.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966206.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966207.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966208.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966209.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966210.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966211.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966212.tif
Created monthly grid! ppt_19

reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966301.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966302.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966303.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966304.tif
Created monthly grid! ppt_196610.tif
Month summed up is: 11
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966305.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966306.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966307.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966308.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966309.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966310.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1966/PPT_1966311.tif
reading file ... s3://dev-et-data/in/DelawareRive

reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967036.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967037.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967038.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967039.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967040.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967041.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967042.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967043.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967044.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967045.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967046.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/1967/PPT_1967047.tif
reading file ... s3://dev-et

...

reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017273.tif
Created monthly grid! ppt_201709.tif
Month summed up is: 10
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017274.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017275.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017276.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017277.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017278.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017279.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017280.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017281.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017282.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2017/PPT_2017283.tif
reading file ... s3://dev-et-data/in/DelawareRive

reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018007.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018008.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018009.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018010.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018011.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018012.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018013.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018014.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018015.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018016.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018017.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018018.tif
reading file ... s3://dev-et

reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018107.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018108.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018109.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018110.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018111.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018112.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018113.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018114.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018115.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018116.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018117.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018118.tif
reading file ... s3://dev-et

reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018207.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018208.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018209.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018210.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018211.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018212.tif
Created monthly grid! ppt_201807.tif
Month summed up is: 8
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018213.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018214.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018215.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018216.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018217.tif
reading file ... s3://dev-et-data/in/DelawareRiver

reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018307.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018308.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018309.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018310.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018311.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018312.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018313.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018314.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018315.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018316.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018317.tif
reading file ... s3://dev-et-data/in/DelawareRiverBasin/PPT/2018/PPT_2018318.tif
reading file ... s3://dev-et

In [8]:
! ls -lh ./junkboxpptsum

total 30M
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:18 ppt_195001.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:18 ppt_195002.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:18 ppt_195003.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195004.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195005.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195006.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195007.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195008.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195009.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195010.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195011.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195012.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195101.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 

In [9]:
! date

Fri Jan 29 14:17:38 UTC 2021


# Create netcdfs as output format

In [10]:
tony_mkdir('./ncdir')

In [11]:
def create_s3_list_of_months(main_prefix, year, output_name='etasw_'):
    the_list = []
    for i in range(1,13):
        month = f'{i:02d}'
        file_object = main_prefix  + str(year) +os.sep+ output_name + str(year) + month + '.tif'
        the_list.append(file_object)
    return the_list

In [12]:
def netcdf_creator(output_name):
    #working_bucket = 'dev-et-data'
    main_prefix = 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/'
    years = range(1950,2019)
    #output_name = 'etasw_'
    #output_name = 'dd_'
    #output_name = 'srf_'

    for year in years:
        my_tifs = create_s3_list_of_months(main_prefix, year, output_name)
        ds = xr_build_cube_concat_ds(my_tifs, output_name)
        nc_file_name = './ncdir' +os.sep+ output_name + str(year) + '.nc'
        ds.to_netcdf(nc_file_name, engine='h5netcdf')

In [13]:
# outs = ['etasw_', 'dd_', 'srf_']

In [14]:
# for out in outs:
#     netcdf_creator(out)

In [15]:
# ! ls ./ncdir

# PPT just wants to be different

In [16]:
def create_local_list_of_months(main_prefix, year, output_name='ppt_'):
    the_list = []
    for i in range(1,13):
        month = f'{i:02d}'
        #file_object = main_prefix  + str(year) + '/' + output_name + str(year) + month + '.tif'
        file_object = main_prefix + output_name + str(year) + month + '.tif'
        the_list.append(file_object)
    return the_list

In [17]:
main_prefix = './junkboxpptsum/'
years = range(1950,2019)
output_name='ppt_sum_'
for year in years:
        my_tifs = create_local_list_of_months(main_prefix, year, output_name)
        ds = xr_build_cube_concat_ds(my_tifs, output_name)
        nc_file_name = './ncdir' +os.sep+ output_name + str(year) + '.nc'
        ds.to_netcdf(nc_file_name, engine='h5netcdf')

./junkboxpptsum/ppt_sum_195001.tif


RasterioIOError: ./junkboxpptsum/ppt_sum_195001.tif: No such file or directory

In [ ]:
! ls ./ncdir

In [ ]:
tony_mkdir('./tarballs')

In [ ]:
#! tar cvfz ./tarballs/wzell_etasw.tgz ./ncdir/eta*.nc
#! tar cvfz ./tarballs/wzell_dd.tgz ./ncdir/dd*.nc
#! tar cvfz ./tarballs/wzell_srf.tgz ./ncdir/srf*.nc
! tar cvfz ./tarballs/wzell_ppt_sum.tgz ./ncdir/ppt_sum*.nc

In [ ]:
! ls ./tarballs

In [ ]:
# copy tar files from tarballs to current dir ()aware
! echo scp -r jupyter-wzell@10.12.68.246:`pwd`/tarballs . 